In [134]:
import subprocess
from multiprocessing import Pool
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [1]:
vol_er = (3.9*0.1*0.1)
vol_cyt = (4.0*0.5*0.5-vol_er)
vol_tot = vol_cyt + vol_er
N_avo=6.0221409e23

In [16]:
def write_mdl(file_name,outfolder,iterations,ic,sf,sm,serca_frac,write=True):
    ftext = """sprintf(my_seed,"%05g",SEED)

    iterations = e3

    INCLUDE_FILE = "../outputLoc.mdl"
    INCLUDE_FILE = "../initialization.mdl"
    INCLUDE_FILE = "parameters.mdl"
    /* Modifications in Parameters */
        VDCC_number_presynaptic = 40

        n_Glu_neuron =  1 //2000
        GluT_density_astrocyte = 0.1 //1000

        ca_conc_presynaptic_er = e-6     //initial ER conc specified here!!!!!
        //ryr_number_presynaptic = 0 //no RYR

    f =                                 //SERCA parameter adjustments here!!!!!!!!!!

    sm = //multiply all serca parameters

    /*Initial SERCA fractions*/
    serca_x1_feq = 
    serca_x1a_feq = 
    serca_x2_feq = 
    serca_y1_feq = 
    serca_y1a_feq = 
    serca_y2_feq = 

    INCLUDE_FILE = "rxn_rate_constants.mdl"
    /* Modifications in Reaction Rate Constants */
    //k_PMCA_0_1 = 0
    //k_PMCA_0_leak = 0


    /* Output Folder */
            //isi = "40"
        sprintf(VDCC, "%g", VDCC_number_presynaptic)
        sprintf(f_mult, "%g", f)   	
        sprintf(init_er_conc, "%g", ca_conc_presynaptic_er*1e6)   	
        fname = 

        output_folder = outputLoc & "ppf/" & fname & "/" & f_mult & "/" & init_er_conc & "/s_" & my_seed

    /* Rates for P/Q-type Voltage Dependent Calcium Channels */
    /* From Bischoffberger et al., 2005, , Biophys. J.*/

        k_VDCC_PQ_C01 = 0
        k_VDCC_PQ_C12 = 0
        k_VDCC_PQ_C23 = 0
        k_VDCC_PQ_C34 = 0
        k_VDCC_PQ_C43 = 0
        k_VDCC_PQ_C32 = 0
        k_VDCC_PQ_C21= 0
        k_VDCC_PQ_C10 = 0
        k_VDCC_PQ_Ca = 0

    INCLUDE_FILE = "triggered_release.inc.mdl"

    INCLUDE_FILE = "rxn_outputRS.mdl"
    //INCLUDE_FILE = "viz_output.mdl"
    """
    ftext=ftext.replace('iterations = ', 'iterations = '+str(iterations))
    ftext=ftext.replace('sm = ', 'sm = '+str(sm))
    ftext=ftext.replace('ca_conc_presynaptic_er = ', 'ca_conc_presynaptic_er = '+str(ic))
    ftext=ftext.replace('f = ','f = '+str(sf))
    ftext=ftext.replace('fname = ','fname = '+'"'+str(outfolder)+'"')
    for i,p in enumerate(['x1','x1a','x2','y2','y1a','y1']):
        old="serca_"+p+"_feq = "
        new="serca_"+p+"_feq = "+str(serca_frac[i])
        ftext=ftext.replace(old,new)
    if write:
        with open(file_name,'w') as wfile:
            wfile.write(ftext)


In [36]:
#write .mdls for sf=4.3 equilibrium at 750uM
sf=4.3
folder="/home/kabir/Project/tripartiteSynapse/brute750_6/"
outfolder="brute6"
initial_conc=[350,450,550,650,700]
serca_frac=[0.5709841 , 0.27425674, 0.06441576, 0.04471076, 0.03791196, 0.00749021]
serca_mult=[3,4,5,6,7,8,9,10]

fnames=[]
#ic=750
iterations=20000
for ic in initial_conc:
    for sm in serca_mult:#[3.5,3.75,4,4.25,4.5,4.75]:
        file_name=folder+"RSnostim_sm"+str(sm)+"_"+str(sf)+"_"+str(ic)+".mdl"
        fnames.append(file_name)
        write_mdl(file_name,outfolder,iterations,ic,sf,sm,serca_frac,write=True)

In [41]:
with open(folder+'run.sh','w') as f:
    for name in fnames:
            f.write("qsub -N "+name.replace('.mdl','').replace(folder,'')+" -v I='"+name.replace(folder,'/home/subhadra/kabir/tripartiteSynapse/brute750_6/')+"' pbs.py\n")

In [42]:
#sync script files to cluster
p=subprocess.call(["rsync","-arhP",folder[:-1],"subhadra@192.168.1.244:/home/subhadra/kabir/tripartiteSynapse/"])

In [177]:
#sync data back to this system
p=subprocess.call(["rsync","-arhP","subhadra@192.168.1.244:/storage/subhadra/kabir/output/ppf/brute6","/data/kabir/output/ppf/"])

In [179]:
%run averaging.ipynb

In [180]:
initial_conc=[350,450,550,650,700]

#averager("/data/kabir/output/ppf/brute6/4.3/350")
sm=4.3
for ic in initial_conc:
    print(sm,ic)
    averager("/data/kabir/output/ppf/brute6/"+str(sm)+"/"+str(ic)+"/")

(4.3, 350)


AttributeError: 'Pool' object has no attribute 'starmap'